- This notebook
  - calls JQUANTS API to get the list of the stocks listed in the Tokyo Stock Exchange.
  - calls Yahoo Finance API to get the latest stock related information such as Price, PE ratio, market cap etc
  - create a CSV file

In [2]:
#import libraries
import requests
import json
import pandas as pd
import yfinance as yf
import datetime as datetime
from tqdm import tqdm
import pandas as pd
from sqlalchemy import create_engine, Column, String, Integer, MetaData, Table
from sqlalchemy.dialects.postgresql import NUMERIC, BIGINT
import psycopg2

# load const
with open('parameters.json', 'r') as file:
    data = json.load(file)

# Define the database connection string (update with your PostgreSQL details)
DATABASE_URI = data["DATABASE_URI"]
mail_address = data["mailaddress"]
password = data["password"]

# Create an engine and metadata object
engine = create_engine(DATABASE_URI)
metadata = MetaData()

In [3]:
data={"mailaddress":mail_address, "password":password}
r_post = requests.post("https://api.jquants.com/v1/token/auth_user", data=json.dumps(data))
rToken = r_post.json()["refreshToken"]
REFRESH_TOKEN = rToken
r_post = requests.post(f"https://api.jquants.com/v1/token/auth_refresh?refreshtoken={REFRESH_TOKEN}")
idToken = r_post.json()["idToken"]

In [21]:
# 上場銘柄一覧
headers = {'Authorization': 'Bearer {}'.format(idToken)}
r = requests.get("https://api.jquants.com/v1/listed/info", headers=headers)

# Pandas DafaFrameへ変換
r_dict = r.json()
df = pd.DataFrame(r_dict['info'])

In [24]:
df["MarketCodeCleansed"] = df[df["MarketCode"].isin(["0111", "0112"])]["Code"].str[:4] + ".T"

In [25]:
ticker = []
ticker = df[df["MarketCode"].isin(["0111", "0112"])]["MarketCodeCleansed"].values.tolist()

In [27]:
df_master_yf = pd.DataFrame(columns=["Code", "previousClose", "trailingPE", "volume", "marketCap", "fiftyTwoWeekLow", "fiftyTwoWeekHigh", "revenuePerShare"])
columns_yf = ["Code", "previousClose", "trailingPE", "volume", "marketCap", "fiftyTwoWeekLow", "fiftyTwoWeekHigh", "revenuePerShare"]
listInfo = ["previousClose", "trailingPE", "volume", "marketCap", "fiftyTwoWeekLow", "fiftyTwoWeekHigh", "revenuePerShare"]

for tk in tqdm(ticker): 
    stock = yf.Ticker(tk)
    data = []
    data.append(tk)
    for info in listInfo:
        try:
            data.append(stock.info[info])
        except:
            data.append("0")     
            
    df_yf_temp = pd.DataFrame(data=[data], columns=columns_yf)
    df_master_yf = pd.concat([df_master_yf, df_yf_temp], ignore_index=True)
#df_yf.head()

  0%|                                                                                                            | 0/3257 [00:00<?, ?it/s]/var/folders/4d/73_4mvkd4nsc69k9dpv182fr0000gn/T/ipykernel_13498/2638317538.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_master_yf = pd.concat([df_master_yf, df_yf_temp], ignore_index=True)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 3257/3257 [05:32<00:00,  9.79it/s]


In [32]:
df_master_yf["MarketCodeCleansed"] = df_master_yf["Code"]
df_master = pd.merge(df, df_master_yf, on='MarketCodeCleansed')
df_master = df_master.drop('Code_y', axis=1)
df_master.dropna(subset=["trailingPE", "marketCap"], inplace=True)
df_master["trailingPE"] = pd.to_numeric(df_master["trailingPE"], errors='coerce')
df_master["marketCap"] = pd.to_numeric(df_master["marketCap"], errors='coerce')
df_short_list = df_master[(df_master["trailingPE"] < 10) & (df_master["marketCap"] < 50000000000) & (df_master["trailingPE"] > 0.5)].sort_values(by=['trailingPE'])

In [289]:
#insert data into PostgresDB
df_short_list.to_sql('t_short_list', con=engine, if_exists='append', index=False)

569